In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
"""
웹 서버 로그 기반 악성 요청 탐지를 위한 분류 모델 구축
로그 데이터 전처리 및 성능 지표(Accuracy, F1 등)를 활용한 모델 평가

웹 서버 로그 데이터를 분석하여 악성 요청을 탐지하는 머신러닝 분류 모델을 구축해야 합니다.
"""

In [80]:
# 데이터 불러오기
df = pd.read_csv('web_server_logs_2.csv')

In [81]:
# 데이터 확인
df.head()

,ip,timestamp,method,status_code,size,label
0,192.168.1.138,2024-11-30 09:26:01,OPTIONS,301,541,0
1,192.168.1.130,2024-11-30 12:00:34,OPTIONS,200,5239,0
2,192.168.1.75,2024-12-01 00:22:12,POST,200,5778,0
3,192.168.1.176,2024-11-30 19:33:26,DELETE,200,3911,0
4,10.0.0.113,2024-11-30 05:37:26,GET,200,7765,0


In [82]:
# timestamp에서 hour 추출
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
df.head()

,ip,timestamp,method,status_code,size,label,hour
0,192.168.1.138,2024-11-30 09:26:01,OPTIONS,301,541,0,9
1,192.168.1.130,2024-11-30 12:00:34,OPTIONS,200,5239,0,12
2,192.168.1.75,2024-12-01 00:22:12,POST,200,5778,0,0
3,192.168.1.176,2024-11-30 19:33:26,DELETE,200,3911,0,19
4,10.0.0.113,2024-11-30 05:37:26,GET,200,7765,0,5


In [83]:
# is_error label 생성
# status_code가 400 이상인 경우 Error로 간주
df['is_error'] = df['status_code'].apply(lambda x: 1 if x >= 400 else 0)
df.tail()

,ip,timestamp,method,status_code,size,label,hour,is_error
1495,192.168.1.65,2024-11-30 05:06:59,GET,503,962,1,5,1
1496,192.168.1.51,2024-11-30 17:12:08,OPTIONS,200,6500,0,17,0
1497,10.0.0.116,2024-11-30 21:50:56,OPTIONS,200,3854,0,21,0
1498,192.168.1.2,2024-11-30 08:23:04,GET,404,4338,1,8,1
1499,10.0.0.1,2024-11-30 21:23:58,DELETE,503,9037,1,21,1


In [84]:
# size 열 표준화
scaler = StandardScaler()
df['size'] = scaler.fit_transform(df[['size']])

In [85]:
# method 열 원-핫 인코딩
df = pd.get_dummies(df, columns=['method'])

In [86]:
# 학습용과 테스트용 데이터 분할
X = df[['hour', 'size', 'method_GET', 'method_POST', 'method_PUT', 'method_DELETE']]
y = df['is_error']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
# Logistic Regression 모델 학습
# 클래스 불균형 문제를 해결하기 위해 class_weight='balanced' 사용
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [89]:
# 모델 평가
y_pred = model.predict(X_test)
print('[Classification Report]')
print('=' * 55)
print(classification_report(y_test, y_pred))

print('모델 평가 결과 분석 : ')
print('Recall : 악성 요청 탐지는 꽤 잘 되고 있으나 정상 요청을 놓치는 경우가 많습니다.')
print('F1-score : 악성 요청은 성능이 양호한 편이지만 정상 요청에 대한 성능 개선이 필요합니다.')

[Classification Report]
              precision    recall  f1-score   support

           0       0.69      0.63      0.66       201
           1       0.36      0.42      0.39        99

    accuracy                           0.56       300
   macro avg       0.52      0.53      0.52       300
weighted avg       0.58      0.56      0.57       300

모델 평가 결과 분석 : 
Recall : 악성 요청 탐지는 꽤 잘 되고 있으나 정상 요청을 놓치는 경우가 많습니다.
F1-score : 악성 요청은 성능이 양호한 편이지만 정상 요청에 대한 성능 개선이 필요합니다.
